In [1]:
# now running many experiments and doing analyses on them all

from neuron import h
from neuron.units import mV, ms
h.load_file('stdrun.hoc')

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import pickle

In [16]:
def gen_histories(sim_length = 1000000, stim_interval = 10, num_histories = 1000):

    # Set up biophysical model
    axon = h.Section(name='axon')
    axon.insert(h.hh)

    # add a synapse
    syn = h.ExpSyn(axon(0))
    syn.tau = 1 * ms
    syn.e = 0 * mV
    syn_current = h.Vector().record(syn._ref_i)

    # add a stimulus
    stim = h.NetStim()
    stim.number = 9999999
    stim.interval = stim_interval * ms
    stim.noise = True
    stim.start = 0 * ms

    stim_times = h.Vector()

    # connect stimulus to synapse
    nc = h.NetCon(stim, syn)
    nc.delay = 0 * ms
    nc.weight[0] = 0.2
    nc.record(stim_times)

    # setup recording
    _t = h.Vector().record(h._ref_t)
    _v = h.Vector().record(axon(0.5)._ref_v)
    _m = h.Vector().record(axon(0.5).hh._ref_m)
    _n = h.Vector().record(axon(0.5).hh._ref_n)
    _h = h.Vector().record(axon(0.5).hh._ref_h)
    spike_times = h.Vector()
    nc_self = h.NetCon(axon(0.5)._ref_v, None, sec=axon)
    nc_self.record(spike_times)

    # run simulation
    h.finitialize(-65 * mV)
    h.continuerun(sim_length * ms)
    
    histories = np.array((_v, _m, _h, _n))[:, random.sample(range(0, len(_v)), num_histories)]

    return histories, np.array((_t, _v, _m, _h, _n, syn_current)), list(spike_times), list(stim_times)

In [3]:
# set up preceding stim times for event horizon experiment (based on output spike)

def gen_preceding_stims(pivot_ind, spike_times, num_history_stims = 20):
    # num_history_stims: this is the number of stims to include in the experiment's history
    # pivot_ind: which output spike to use as pivot

    # choose output spike as pivot
    pivot = spike_times[pivot_ind]

    # isolate the stimuli preceding the pivot spike
    preceding_stims = [st for st in stim_times if st < pivot]
    # check to make sure there are enough stims here
    if len(preceding_stims) < num_history_stims:
        return False, False
    preceding_stims = preceding_stims[-num_history_stims:]

    # set the last stimuli before the pivot spike as t0
    pivot = pivot - preceding_stims[-1]
    preceding_stims = [st - preceding_stims[-1] for st in preceding_stims]

    # round stimuli and pivot to the nearest 1/40th ms
    preceding_stims = [(math.floor(ps * 40))/40 for ps in preceding_stims]
    pivot = (math.floor(pivot * 40))/40
    
    return preceding_stims, pivot

def view_preceding_stimuli(preceding_stims, pivot):
    # view the preceding stimuli
    plt.figure(figsize = (15, 3))
    plt.vlines(preceding_stims, 0, 1, color = 'black')
    plt.vlines(pivot, 0, 1, color = 'red')
    plt.xlabel('time (ms)')
    plt.yticks([])
    plt.show()
    
    return

In [4]:
# initialize simulations with new initial conditions and custom stimuli
def sim_init(axon, _v_init, _m_init, _h_init, _n_init):
    for seg in axon:
        seg.v = _v_init
        seg.hh.m = _m_init
        seg.hh.h = _h_init
        seg.hh.n = _n_init

def seed_sim(history, stimuli, sim_length = 100):
    
    # Set up biophysical model
    axon = h.Section(name='axon')
    axon.insert(h.hh)
    
    # add a synapse
    syn = h.ExpSyn(axon(0))
    syn.tau = 1 * ms
    syn.e = 0 * mV
    syn_current = h.Vector().record(syn._ref_i)
    
    # add custum stimuli
    netstims = [h.NetStim() for stimulus in stimuli]
    netcons = [] # store netcons here as they need to exist in memory to parallelize
    for netstim, stimulus in zip(netstims, stimuli):
        netstim.number = 1
        netstim.start = stimulus
        netcon = h.NetCon(netstim, syn)
        netcon.weight[0] = 0.2
        netcons.append(netcon)
        
        netcon.delay = 0 * ms 

    stim_times = h.Vector()

    # setup recording
    _t = h.Vector().record(h._ref_t)
    _v = h.Vector().record(axon(0.5)._ref_v)
    _m = h.Vector().record(axon(0.5).hh._ref_m)
    _n = h.Vector().record(axon(0.5).hh._ref_n)
    _h = h.Vector().record(axon(0.5).hh._ref_h)
    
    spike_times = h.Vector()
    nc_self = h.NetCon(axon(0.5)._ref_v, None, sec=axon)
    nc_self.record(spike_times)

    # run simulation
    
    # initialize simulation
    _v_init = history[0]
    _m_init = history[1]
    _h_init = history[2]
    _n_init = history[3]
    
    fih = h.FInitializeHandler((sim_init,(axon, _v_init, _m_init, _h_init, _n_init)))
    h.finitialize()
    h.continuerun(sim_length * ms)
    
    return np.array((_t, _v, _m, _h, _n, syn_current)), list(spike_times)

def run_simulation(stimuli, histories, extended_duration = 20):
    # function to run a simulation with a given set of histories, a set of predetermined stimuli, and an extended duration
    # extended_duration: amount of time after the last stimulus to run the experiment
    
    exp_out = []
    
    for i in range(histories.shape[1]):
        history = histories[:,i]
        sim_out, spike_times = seed_sim(history, stimuli, sim_length = extended_duration + stimuli[-1])
        exp_out.append(spike_times)
        
    return exp_out

def preceding_stims_walk(preceding_stims, histories):
    # function to walk through each of the preceding stimuli
    resulting_NSTs = [] # next-spike-times
    resulting_PSTs = [] # pre-spike-times (those spikes that occur before the last stimulus)

    for num_stims_included in range(1, len(preceding_stims) + 1):
        current_stims = preceding_stims[len(preceding_stims) - num_stims_included:]

        # set the first stim to t0
        current_stims = [cs - current_stims[0] for cs in current_stims]

        #print('number of preceding stims:', len(current_stims))
        exp_out = run_simulation(current_stims, histories)
        NSTs = [] # next-spike-times
        PSTs = [] # pre-spike-times (those spikes that occur before the last stimulus)

        # process each exp_out
        for spike_times in exp_out:
            # separate spike_times before and after the last stimulus
            pre_spike_times = [sp for sp in spike_times if sp <= current_stims[-1]]
            post_spike_times = [sp for sp in spike_times if sp > current_stims[-1]]
            
            #print(len(spike_times), len(pre_spike_times), len(post_spike_times))

            if len(post_spike_times) > 0:
                NSTs.append(post_spike_times[0])
            else:
                NSTs.append('na') # did not spike
                
            PSTs += pre_spike_times 


        # set the last stimulus to t0
        _NSTs = []
        for nst in NSTs:
            if nst != 'na':
                nst = nst - current_stims[-1]
            _NSTs.append(nst)
        
        PSTs = [pst - current_stims[-1] for pst in PSTs]
            
        #NSTs = [nst - current_stims[-1] for nst in NSTs if nst != 'na']    
        #print('number of NSTs:', len(NSTs))

        resulting_NSTs.append(_NSTs)
        resulting_PSTs.append(PSTs)
        
    return resulting_NSTs, resulting_PSTs

In [5]:
# equations for the time constants

def vtrap(x,y):
    if abs(x/y) < 1e-6:
        return y * (1 - x/y/2)
    else:
        return x / (math.exp(x / y) - 1)
        
def calc_mtau(v, q10):
    alpha = .1 * vtrap(-(v + 40), 10)
    beta =  4 * math.exp(-(v + 65) / 18)
    _sum = alpha + beta
    mtau = 1/(q10 * _sum)
    minf = alpha / _sum
    return mtau, minf

def calc_htau(v, q10):
    alpha = .07 * math.exp(-(v+65) / 20)
    beta = 1 / (math.exp(-(v+35) / 10) + 1)
    _sum = alpha + beta
    htau = 1/(q10 * _sum)
    hinf = alpha / _sum
    return htau, hinf

def calc_ntau(v, q10):
    alpha = .01 * vtrap(-(v + 55), 10) 
    beta = .125 * math.exp(-(v + 65) / 80)
    _sum = alpha + beta
    ntau = 1 / (q10 * _sum)
    ninf = alpha / _sum
    return ntau, ninf

In [19]:
histories, sim_df, spike_times_initial, stim_times = gen_histories(
    sim_length = 1000000, 
    num_histories = 1000)

In [20]:
len(spike_times_initial)

21768

## Experiment

In [14]:
# generate histories

# long run
histories, sim_df, spike_times_initial, stim_times = gen_histories(
    sim_length = 10000, 
    num_histories = 1000)

# short run
#histories, sim_df, spike_times_initial, stim_times = gen_histories(sim_length = 2000, num_histories = 10)

print('number of histories:', histories.shape[1])
print('number of spikes: ', len(spike_times_initial))

number of histories: 1000
number of spikes:  205


In [7]:
# calculate the time constants
q10 = math.pow(3, ((h.celsius - 6.3) / 10)) # some important variable

taus = []
infs = []
for i in range(sim_df.shape[1]):
    mtau, minf = calc_mtau(sim_df[1,i], q10)
    htau, hinf = calc_htau(sim_df[1,i], q10)
    ntau, ninf = calc_ntau(sim_df[1,i], q10)
    
    taus.append([mtau, htau, ntau])
    infs.append([minf, hinf, ninf])

taus = np.array(taus).T
infs = np.array(infs).T

In [12]:
# save experiment results to files

# model state variables
path_sim_df = './data.2/sim_df'
np.save(path_sim_df, np.array(sim_df))

# stimuli
path_stim_times = './data.2/stim_times.txt'
with open(path_stim_times, 'w') as writer:
    for stim in stim_times:
        writer.write("%s\n" % stim)

# initial spike times
path_spike_times_initial = './data.2/spike_times_initial.txt'
with open(path_spike_times_initial, 'w') as writer:
    for spike in spike_times_initial:
        writer.write("%s\n" % spike)

# histories
np.save('./data.2/histories', np.array(histories))

# time constants
np.save('./data.2/taus', np.array(taus))
np.save('./data.2/infs', np.array(infs))

In [13]:
# generate preceding_stims sets

'''
# pick the pivot inds to use
num_preceding_stims_sets = 100
pivot_inds = random.sample(range(10, len(spike_times_initial)), num_preceding_stims_sets)
'''

# data dirs
pstims_ext = './data.2/preceding_stims/'
rNSTs_ext = './data.2/resulting_NSTs/'
rPSTs_ext = './data.2/resulting_PSTs/'

pivots = {}

for i, pivot_ind in enumerate(range(len(spike_times_initial))):
    # generate the preceding stims
    preceding_stims, pivot = gen_preceding_stims(
        pivot_ind = pivot_ind, 
        spike_times = spike_times_initial)
    
    # select all pivots that apply (have at least 10 preceding stimuli)
    if preceding_stims == False:
        #print(i, 'th pivot not acceptable')
        pass
    else:
        pivots[pivot_ind] = pivot

        # save preceding_stims to file
        pstims_file = 'pstims' + str(i) + '.txt'
        with open(pstims_ext + pstims_file, 'w') as writer:
            for stim in preceding_stims:
                writer.write("%s\n" % stim)

        # run the preceding stims walk
        resulting_NSTs, resulting_PSTs = preceding_stims_walk(preceding_stims, histories)

        # save resulting_NSTs to file
        rNSTs_file = 'rNSTs' + str(i)
        rPSTs_file = 'rPSTs' + str(i) + '.pkl'
        np.save(rNSTs_ext + rNSTs_file, np.array(resulting_NSTs))
        with open(rPSTs_ext + rPSTs_file, 'wb') as f:
            pickle.dump(resulting_PSTs, f)
        
        if (i+1) % 10 == 0:
            print(i+1, '/', len(spike_times_initial))
            
# save pivots to file
with open('./data.2/pivots.pkl', 'wb') as f:
    pickle.dump(pivots, f)

10 / 208
20 / 208
30 / 208
40 / 208
50 / 208
60 / 208
70 / 208
80 / 208
90 / 208
100 / 208
110 / 208
120 / 208
130 / 208
140 / 208
150 / 208
160 / 208
170 / 208
180 / 208
190 / 208
200 / 208


In [ ]:
# takes about an hour or so to run